# How can we use parameterized decorators?

- In previous lessons, we've already used parameterized decorators
    - E.g. `@wraps(fn)`, `@lru_cache(maxsize=8)`, etc.

- However, we haven't created any parameterized decorators
    - E.g. `@timed`, `@logged`, etc.

- Since parameterized decorators have the parentheses at the end, they're a function call
    - However, since our unparameterized decorators don't, they're not a function call

- Let's review our `timed` decorator

In [2]:
def timed(fn):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            elapsed = end - start
            total_elapsed += elapsed
            
        avg_elapsed = total_elapsed / 10
        print(avg_elapsed)
        
        return result
    return inner

- Now, if we define some function `my_func`, we can decorate it two ways:


**`@` Method**
```python
@timed
def my_func():
    # Do something
```


**Other Method**
```python
def my_func():
    # Do something
    
my_func = timed(my_func)
```

# How can we convert the hard-coded 10 in our function to a parameter?

- We'll try adding it to the `timed` decorator

In [3]:
def timed(fn, n_reps):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        
        for i in range(n_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            elapsed = end - start
            total_elapsed += elapsed
            
        avg_elapsed = total_elapsed / n_reps
        print(avg_elapsed)
        
        return result
    return inner

- Now, the way to decorate `my_func` is:

```python
my_func = timed(my_func, 10)
```

- *But what about using `@`?*
    - **Won't work**:

```python
@timed(10)
def my_func():
    # Do something
```

- `timed` expects two parameters
    - We need to rethink the solution

- **Recall**: `timed` is a function that returns the `inner` closure
    - i.e. combines the original function `fn` with elements outside its scope

- If we want `@timed(10)` to work, then `timed(10)` needs to return a decorator
    - i.e. needs to return an instance of `timed` instead of `inner`

- It would need to be something like:

```python
dec = timed(10)

@dec
def my_func():
    # Do something
```

- Therefore, **`timed` is no longer a decorator in this method**
    - It's a **decorator generating function**
        - This is called a **decorator factory**

In [4]:
def outer(n_reps):
    def timed(fn):
        from time import perf_counter
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(n_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                elapsed = end - start
                total_elapsed += elapsed
            avg_elapsed = total_elapsed / n_reps
            print(avg_elapsed)
            return result
        return inner
    return timed

- Now, if we call `outer(10)`, it'll return a decorator `timed` with `n_reps = 10`
    - i.e. it'll return our initial decorator we defined with the hard-coded 10

- So, the new way of decorating `my_func` would be:

```python
my_func = outer(10)(my_func)
```

____

- **Recall**: the following two ways to decorate `my_func` with our old version of `timed` are equivalent:

**`@` Method**
```python
@timed
def my_func():
    # Do something
```


**Other Method**
```python
def my_func():
    # Do something
    
my_func = timed(my_func)
```

- As we can see, to translate the Other Method to the `@` Method, we just:
    1. Took the `timed(my_func)` line
    2. Removed the parameter `my_func` (i.e. parentheses)
    3. Put it above the function with the `@` symbol
____

- So, with our new line `my_func = outer(10)(my_func)`, the equivalent would be:

```python
@outer(10)
def my_func():
    # Do something
```

- Boom!
    - We're done!

____

# Examples

In [5]:
def dec_factory():
    print('running dec_factory')
    
    def dec(fn):
        print('running dec')
        
        def inner(*args, **kwargs):
            print('running inner')
            return fn(*args, **kwargs)
        
        return inner
    
    return dec

In [6]:
dec = dec_factory()

running dec_factory


- As we can see, the decorator factory was used to create a decorator

In [7]:
@dec
def my_func():
    print('running my func')

running dec


- Then, the decorator we created was used to decorate `my_func`

In [8]:
my_func()

running inner
running my func


- Finally, when we call `my_func`, it runs both `inner` and `my_func`
    - This is because decorating `my_func` is equivalent to setting `my_func = dec(my_func)`, and `dec(my_func)` returns an instance of `inner`

- An alternative way to equivalently decorate our function is:

In [10]:
@dec_factory()
def my_func():
    print('running my func')

running dec_factory
running dec


- In this example, it runs `decorator_factory`, then decorates `my_func` in a single step